In [1]:
import tensorflow as tf
import requests
import pandas as pd
import base64
import json

In [2]:
df = pd.read_csv("data/Mental-Health-Twitter-CleanData.csv")
df.head()

,post_text,label
0,its just over years since i was diagnosed wit...,1
1,its sunday i need a break so im planning to sp...,1
2,awake but tired i need to sleep but my brain h...,1
3,rt sewhq retro bears make perfect gifts and ar...,1
4,its hard to say whether packing lists are maki...,1


In [3]:
test_true = df['post_text'][0]
test_false = df['post_text'][19999]
print(test_true)
print(test_false)

its just over  years since i was diagnosed with anxiety and depression today im taking a moment to reflect on how far ive come since
black holes are where god is dividing by zero


In [4]:
def prepare_json(text):
    feature_spec = {
        "post_text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })


In [5]:
def get_prediction(input):
    json_data = prepare_json(input)
    endpoint = "http://103.190.215.45:8501/v1/models/mental-health-twitter-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json().get("predictions")
    if prediction:
        prediction_value = prediction[0][0]
        return "Depression" if prediction_value > 0.7 else "Not Depressed"
    else:
        return "Error in prediction"

In [6]:
print(f"Test True Depression : {get_prediction(test_true)}")
print(f"Test False Depression : {get_prediction(test_false)}")

Test True Depression : Depression
Test False Depression : Not Depressed
